In [1]:
%pylab inline
%config InlineBackend.figure_format = 'svg' 

Populating the interactive namespace from numpy and matplotlib


In [2]:
def plotRoc (cv, classifier, X, y, name = None):
    import pylab as pl
    from scipy import interp
    from sklearn.metrics import roc_curve, auc




    ###############################################################################
    # Classification and ROC analysis
    if not name: 
        name = str(classifier).split('(')[0]
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []
    
    data = []

    for i, (train, test) in enumerate(cv):
        print "#### ITERATION", i, "train ####"
        X_train = X[train]
        
        print "#### ITERATION", i, "test ####"
        X_test = X[test]
        
        probas_ = classifier.fit(X_train, y[train]).predict_proba(X_test)
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        pl.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
        data.append({"x": fpr,
                     "y": tpr,
                     "type": "scatter",
                     "mode": "lines+markers",
                     "name": "Fold %s: %0.2f"%(i, roc_auc),
                     "line": {"dash" :"dot",
                              "width" :1},
                     "opacity": 0.5})
        
    pl.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
    data.append({"x": [0, 1],
                 "y": [0, 1],
                 "type": "scatter",
                 "mode": "lines",
                 "name": "Luck",
                 "line": {"dash" :"dash",
                          "width" :2,
                          "color" :"grey"}})
    
    mean_tpr /= len(cv)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    pl.plot(mean_fpr, mean_tpr, 'k--',
            label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)


    data.insert(0,{"x": mean_fpr,
                   "y": mean_tpr,
                   "type": "scatter",
                   "mode": "lines",
                   "name": "%s AUC: %0.2f"%(name, mean_auc),
                   "line": {"dash": "dash",
                            "width": 4,
                            "color": "black"}})

    pl.xlim([-0.05, 1.05])
    pl.ylim([-0.05, 1.05])
    pl.xlabel('False Positive Rate')
    pl.ylabel('True Positive Rate')
    pl.title('Receiver operating characteristic example')
    pl.legend(loc="lower right")
    pl.show()

#     plot(data)
    return data

In [3]:
import pandas as pd
X = pd.read_csv("train_set.csv")
y = pd.read_csv("train_label.csv")

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(X)
print pca.explained_variance_ratio_

[ 0.14004552  0.08828155  0.06629792  0.05064028  0.04656597  0.0377272
  0.03308945  0.0293897   0.02403482  0.02341814]


In [6]:
labels = pd.read_csv('../sm-labeled.csv', index_col=0)

In [7]:
matrix = pd.read_csv('../expression-all-labeled-transposed.csv', index_col=0)

In [73]:
tsne = labels[["1", "2"]]

In [10]:
pca.fit(matrix)
print pca.explained_variance_ratio_

[ 0.17260515  0.09790987  0.06711368  0.05311525  0.03443384  0.0336625
  0.02746045  0.02449515  0.02011884  0.01872892]


In [21]:
import sklearn.cluster

In [23]:
kmeans = sklearn.cluster.KMeans(n_clusters=4)

In [74]:
fit = kmeans.fit(tsne)

In [75]:
labels['py_cluster'] = fit.labels_ + 1

In [76]:
labels['y_WNT'] = labels.subtype_original\
        .replace("WNT", 1)\
        .replace("Group 3", 0)\
        .replace("SHH", 0)\
        .replace("Group 4", 0)
labels['y_SHH'] = labels.subtype_original\
        .replace("WNT", 0)\
        .replace("Group 3", 0)\
        .replace("SHH", 1)\
        .replace("Group 4", 0)
labels['y_G3'] = labels.subtype_original\
        .replace("WNT", 0)\
        .replace("Group 3", 1)\
        .replace("SHH", 0)\
        .replace("Group 4", 0)
labels['y_G4'] = labels.subtype_original\
        .replace("WNT", 0)\
        .replace("Group 3", 0)\
        .replace("SHH", 0)\
        .replace("Group 4", 1)
labels['y_r1'] = labels.cluster.replace(1,1).replace(2,0).replace(3,0).replace(4,0)
labels['y_r2'] = labels.cluster.replace(1,0).replace(2,1).replace(3,0).replace(4,0)
labels['y_r3'] = labels.cluster.replace(1,0).replace(2,0).replace(3,1).replace(4,0)
labels['y_r4'] = labels.cluster.replace(1,0).replace(2,0).replace(3,0).replace(4,1)
labels['y_py1'] = labels.py_cluster.replace(1,1).replace(2,0).replace(3,0).replace(4,0)
labels['y_py2'] = labels.py_cluster.replace(1,0).replace(2,1).replace(3,0).replace(4,0)
labels['y_py3'] = labels.py_cluster.replace(1,0).replace(2,0).replace(3,1).replace(4,0)
labels['y_py4'] = labels.py_cluster.replace(1,0).replace(2,0).replace(3,0).replace(4,1)

In [77]:
for clust in "y_r1", "y_r2", "y_r3", "y_r4":
    for orig in "y_SHH", "y_WNT", "y_G3", "y_G4":
        if labels[clust].corr(labels[orig]) > 0:
            print clust, orig, labels[clust].corr(labels[orig])

y_r1 y_WNT 0.718695908595
y_r2 y_G3 0.711914880617
y_r3 y_SHH 0.870912244612
y_r4 y_G4 0.836544374764


In [78]:
for clust in "y_py1", "y_py2", "y_py3", "y_py4":
    for orig in "y_SHH", "y_WNT", "y_G3", "y_G4":
        if labels[clust].corr(labels[orig]) > 0: 
            print clust, orig, labels[clust].corr(labels[orig]) 

y_py1 y_SHH 0.870912244612
y_py2 y_G4 0.844959097239
y_py3 y_WNT 0.718695908595
y_py4 y_G3 0.721458258204


In [132]:
original_subtype_centroids = labels.groupby("subtype_original")[["1","2"]].mean()

In [130]:
fit.cluster_centers_

array([[ 64.29040324,  73.56053651],
       [-53.72031926, -75.09438454],
       [-53.27673037,  67.82164631],
       [ 56.75719515, -44.23675708]])

In [110]:
labels["centroid_1"] = None
labels["centroid_2"] = None

In [131]:
labels["orig_subtype_centroid_1"] = None
labels["orig_subtype_centroid_2"] = None

In [143]:
labels.columns

Index([u'study', u'platform', u'sample', u'class', u'subtype_original',
       u'gender', u'histology', u'1', u'2', u'cluster', u'py_cluster',
       u'y_WNT', u'y_SHH', u'y_G3', u'y_G4', u'y_r1', u'y_r2', u'y_r3',
       u'y_r4', u'y_py0', u'y_py1', u'y_py2', u'y_py3', u'y_py4',
       u'centroid_1', u'centroid_2', u'orig_subtype_centroid_1',
       u'orig_subtype_centroid_2'],
      dtype='object')

In [150]:
for cluster in original_subtype_centroids.index:
    print cluster
    labels.orig_subtype_centroid_1[labels.subtype_original == cluster] = original_subtype_centroids.ix[cluster][0]
    labels.orig_subtype_centroid_2[labels.subtype_original == cluster] = original_subtype_centroids.ix[cluster][1]


/Users/James/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/James/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Group 3
Group 4
SHH
WNT


In [151]:
for i in range(4):
    cluster = i + 1
    labels.centroid_1[labels.py_cluster == cluster] = fit.cluster_centers_[i][0]
    labels.centroid_2[labels.py_cluster == cluster] = fit.cluster_centers_[i][1]

/Users/James/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/James/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [167]:
labels[["subtype_original", "py_cluster", "1", "2", "centroid_1", "centroid_2", "orig_subtype_centroid_1", "orig_subtype_centroid_2"]]

,subtype_original,py_cluster,1,2,centroid_1,centroid_2,orig_subtype_centroid_1,orig_subtype_centroid_2
GSM529305,SHH,1,29.725065,120.365592,64.2904,73.56054,52.53207,68.95395
GSM529306,Group 3,4,40.749296,-15.092528,56.7572,-44.23676,36.04151,-19.68945
GSM529307,SHH,1,30.693780,29.252512,64.2904,73.56054,52.53207,68.95395
GSM529308,SHH,1,52.832521,70.384884,64.2904,73.56054,52.53207,68.95395
GSM529309,Group 4,2,-98.294091,-21.346516,-53.72032,-75.09438,-44.22623,-76.67182
GSM529310,SHH,1,75.799328,108.134862,64.2904,73.56054,52.53207,68.95395
GSM529311,SHH,1,53.442551,118.179978,64.2904,73.56054,52.53207,68.95395
GSM529312,WNT,3,-51.765966,117.382401,-53.27673,67.82165,-68.49265,69.16468
GSM529313,SHH,1,98.366602,84.196291,64.2904,73.56054,52.53207,68.95395
GSM529314,Group 4,2,-3.553620,-42.875751,-53.72032,-75.09438,-44.22623,-76.67182


In [168]:
new_centroids = labels[["subtype_original", "py_cluster", "1", "2", "centroid_1", "centroid_2", "orig_subtype_centroid_1", "orig_subtype_centroid_2"]]

In [169]:
new_centroids.to_csv("new-centroids.csv")

In [166]:
new-centroids = labels[["subtype_original", "py_cluster", "1", "2", "centroid_1", "centroid_2", "orig_subtype_centroid_1", "orig_subtype_centroid_2"]]")

SyntaxError: EOL while scanning string literal (<ipython-input-166-f58f73142022>, line 1)

In [164]:
from sklearn.metrics import mean_squared_error
for cluster, df in labels.groupby("py_cluster"):
    tsne_real = df[["1", "2"]]
    centroid = df[["centroid_1", "centroid_2"]]
    print cluster, mean_squared_error(tsne_real, centroid)

1 1093.49286441
2 1289.2115111
3 1146.55506761
4 1754.62683398


In [163]:
for cluster, df in labels.groupby("subtype_original"):
    tsne_real = df[["1", "2"]]
    centroid = df[["orig_subtype_centroid_1", "orig_subtype_centroid_2"]]
    print cluster, mean_squared_error(tsne_real, centroid)

Group 3 3596.83245341
Group 4 1728.65861245
SHH 1236.61203981
WNT 1441.11588896


In [96]:
y_true.columns = "first", "second"

In [101]:
y_true['first_centroid'] = fit.cluster_centers_[i][0]

In [100]:
y_true

,first,second,first_pred
GSM529306,40.749296,-15.092528,56.757195
GSM529321,72.655164,-57.785986,56.757195
GSM529336,40.317096,-78.359991,56.757195
GSM529339,89.893270,-32.917533,56.757195
GSM529342,181.608241,-9.615965,56.757195
GSM529344,17.156591,-47.492000,56.757195
GSM529346,135.571096,-19.684244,56.757195
GSM529355,94.110329,-8.780783,56.757195
GSM529358,17.876498,-1.705580,56.757195
GSM529362,64.326924,-131.392161,56.757195


In [ ]:
kf = KFold(len(labels.index), shuffle=True, n_folds=3)
sv = SVC(kernel = "linear", probability=True, random_state=0)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_SHH.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_c3.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_WNT.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_c1.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_G3.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_c2.values)

In [ ]:
sum(labels.y_c2.values), sum(labels.y_G3.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_G4.values)

In [ ]:
data = plotRoc(kf, sv, tsne.values, labels.y_c4.values)

In [ ]:
matrix.shape